In [ ]:
!wget https://github.com/adoptium/temurin11-binaries/releases/download/jdk-11.0.24%2B8/OpenJDK11U-jdk_x64_linux_hotspot_11.0.24_8.tar.gz
!tar xzf OpenJDK11U-jdk_x64_linux_hotspot_11.0.24_8.tar.gz
!rm OpenJDK11U-jdk_x64_linux_hotspot_11.0.24_8.tar.gz
!wget https://archive.apache.org/dist/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xzf spark-3.5.2-bin-hadoop3.tgz
!rm spark-3.5.2-bin-hadoop3.tgz
!pip install findspark

In [ ]:
import os
import sys
import subprocess

working_dir = subprocess.run(['pwd'], stdout = subprocess.PIPE).stdout.strip().decode("utf-8")
print(working_dir)
os.environ["JAVA_HOME"] = working_dir + "/jdk-11.0.24+8/"
os.environ["SPARK_HOME"] = working_dir + "/spark-3.5.2-bin-hadoop3/"
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9.7-src.zip")

number_cores = 2
memory_gb = 6

In [ ]:
import findspark
import pyspark

findspark.init()

conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))

sc = pyspark.SparkContext(conf=conf)

In [ ]:
# Installs pybaseball for Statcast data
!pip install pybaseball --quiet

# Imports Statcast scraper
from pybaseball import statcast
import pandas as pd

print("Downloading 2022 data...")
data_2022 = statcast(start_dt="2022-04-07", end_dt="2022-10-02")

print("Downloading 2023 data...")
data_2023 = statcast(start_dt="2023-03-30", end_dt="2023-10-01")

print("Downloading 2024 data... ")
data_2024 = statcast(start_dt="2024-03-28", end_dt="2024-09-29")

combined = pd.concat([data_2022, data_2023, data_2024])

# Saves the data to a CSV. May take some time (took me around 13 minutes)
combined.to_csv("statcast_2022_2024.csv", index=False)
print("Saved as statcast_2022_2024.csv")

# Preview row count (Spoiler alert: it should be 2,121,390)
print(f"Total rows: {combined.shape[0]}")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, desc

spark = SparkSession(sc)

# You should have statcast_2022_2024.csv in your files
df = spark.read.option("header", True).csv("statcast_2022_2024.csv")

df.printSchema()
print(f"Total rows: {df.count()}")
df.show(5)

In [ ]:
df_cleaned = df.drop("player_name")

In [ ]:
from pybaseball import playerid_reverse_lookup
import pandas as pd

# Gets unique batter IDs
batter_ids = df_cleaned.select("batter").distinct().toPandas()
batter_ids = batter_ids["batter"].dropna().astype(int).tolist()

# Reverse lookup to get names
lookup_df = playerid_reverse_lookup(batter_ids, key_type='mlbam')
lookup_df['batter'] = lookup_df['key_mlbam']
lookup_df['player_name'] = lookup_df['name_first'] + ' ' + lookup_df['name_last']

# Renames player_name
lookup_df = lookup_df.rename(columns={"player_name": "batter_name"})

lookup_df = lookup_df[['batter', 'batter_name']]

In [ ]:
lookup_spark_df = spark.createDataFrame(lookup_df)
df_with_names = df.drop("player_name").join(lookup_spark_df, on="batter", how="left")

## Top 10 Hitters by Average Exit Velocity

In this cell, I calculated the average exit velocity for each hitter and show the top 10. This gives a quick look at which hitters generate the most power off the bat.

In [ ]:
from pyspark.sql.functions import avg, count, desc, col

batted_balls = df_with_names.filter(
    (col("launch_speed").isNotNull()) &
    (col("events").isNotNull()) &
    (col("batter_name").isNotNull())
)

# Calculates exit velocity and batted ball count
exit_velocities = (
    batted_balls.groupBy("batter_name")
                .agg(
                    count("*").alias("batted_ball_count"),
                    avg("launch_speed").alias("avg_exit_velocity")
                )
)

# Filters for qualified hitters (250+ tracked batted balls)
qualified_hitters = (
    exit_velocities.filter(col("batted_ball_count") >= 250)
                   .orderBy(desc("avg_exit_velocity"))
)

qualified_hitters.show(10)

+-----------------+-----------------+-----------------+
|      batter_name|batted_ball_count|avg_exit_velocity|
+-----------------+-----------------+-----------------+
|      aaron judge|             1019|96.42070657507358|
|    shohei ohtani|             1244|94.36543408360122|
|giancarlo stanton|              780| 94.2775641025641|
|       oneil cruz|              582| 94.0618556701031|
|   yordan álvarez|             1144|93.67910839160844|
|     ronald acuña|             1034|93.21344294003866|
|    rafael devers|             1271| 93.0765538945712|
|     matt chapman|             1125|92.95884444444444|
|   kyle schwarber|             1131|92.90919540229893|
|vladimir guerrero|             1543|92.85813350615682|
+-----------------+-----------------+-----------------+
only showing top 10 rows



## Most Common Pitch Types

Here I counted how many times each pitch type was thrown across the entire dataset. This shows how pitchers attack hitters. For example, whether fastballs are still dominant or if breaking balls are more common.


In [ ]:
pitch_type_counts = (
    df.groupBy("pitch_type")
      .agg(count("*").alias("count"))
      .orderBy(desc("count"))
)

pitch_type_counts.show()

+----------+------+
|pitch_type| count|
+----------+------+
|        FF|687060|
|        SL|342236|
|        SI|329214|
|        CH|227934|
|        FC|166039|
|        CU|143574|
|        ST|115196|
|        FS| 49192|
|        KC| 43557|
|        SV|  8983|
|        FA|  3011|
|        EP|  1636|
|        KN|  1287|
|      NULL|   942|
|        FO|   928|
|        CS|   236|
|        SC|   227|
|        PO|   138|
+----------+------+



## Top 10 Pitchers by Average 4-Seam Fastball Velocity

This section filters for 4-seam fastballs and calculates the average velocity for each pitcher. I show the top 10 hardest throwers in the league based on Statcast data.

In [ ]:
fastest_fastballs = (
    df.filter((col("pitch_type") == "FF") & col("release_speed").isNotNull())
      .groupBy("pitcher")
      .agg(avg("release_speed").alias("avg_fastball_speed"))
      .orderBy(desc("avg_fastball_speed"))
      .limit(10)
)

pitcher_names = (
    df.select("pitcher", "player_name")
      .filter(col("player_name").isNotNull())
      .dropDuplicates(["pitcher"])
)

fastest_fastballs_named = (
    fastest_fastballs.join(pitcher_names, on="pitcher", how="left")
                     .select("player_name", "avg_fastball_speed")
)

fastest_fastballs_named.show()

+----------------+------------------+
|     player_name|avg_fastball_speed|
+----------------+------------------+
|   Miller, Mason| 99.96862348178138|
|   Helsley, Ryan| 99.61642475171885|
|      Joyce, Ben|101.64913151364762|
| Clase, Emmanuel|100.11333333333333|
| Halvorsen, Seth| 99.95353535353533|
| Bautista, Félix| 99.71935483870968|
|    Uribe, Abner|100.82812500000001|
|   Doval, Camilo| 99.56666666666666|
|    Duran, Jhoan|101.06242424242424|
|Martinez, Justin|100.54857142857144|
+----------------+------------------+



## Batted Ball Outcome Distribution

This table shows the most common events in the dataset like singles, home runs, strikeouts, etc. It provides an idea of how frequently different outcomes occur when the ball is put in play.

In [ ]:
event_counts = (
    df.filter(col("events").isNotNull())
      .groupBy("events")
      .agg(count("*").alias("count"))
      .orderBy(desc("count"))
)

event_counts.show()

+--------------------+------+
|              events| count|
+--------------------+------+
|           field_out|218503|
|           strikeout|122520|
|              single| 77271|
|                walk| 43740|
|              double| 23797|
|            home_run| 16431|
|           force_out| 10825|
|grounded_into_dou...|  9980|
|        hit_by_pitch|  6124|
|             sac_fly|  3630|
|         field_error|  3290|
|              triple|  2041|
|            sac_bunt|  1262|
|     fielders_choice|  1129|
|         double_play|  1117|
|        truncated_pa|   879|
| fielders_choice_out|   875|
|strikeout_double_...|   330|
|      catcher_interf|   270|
| sac_fly_double_play|    59|
+--------------------+------+
only showing top 20 rows

